In [1]:
from astropy.table import Table, vstack
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as au
import glob

In [2]:
# Define names and filenames...
root_dir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies'

sample_table_file = '%s/data_misc/sample_table/phangs_sample_table_v1p6.fits' %root_dir
muscat_table_file = '%s/data_misc/nebulae_catalogue_v3/Nebulae_catalogue_v3.fits' %root_dir

print(sample_table_file)
print(muscat_table_file)

sample_table = Table.read(sample_table_file)
muscat_table = Table.read(muscat_table_file)

/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_misc/sample_table/phangs_sample_table_v1p6.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_misc/nebulae_catalogue_v3/Nebulae_catalogue_v3.fits


In [3]:
sample_table_ = []
gals = np.unique(muscat_table['gal_name'])

for i, gal in enumerate(gals):
    mask = sample_table['name'] == gal.lower()
    sample_table_ += sample_table[mask]
    
sample_table = vstack(sample_table_)

In [4]:
gals = np.unique(muscat_table['gal_name'])
met = np.empty(len(gals)) *au.dimensionless_unscaled

for i, gal in enumerate(gals): 
    mask = muscat_table['gal_name']==gal
    # rmin_as[i] = np.nanmin((table[mask]['region_circ_rad']).quantity)
    # rmin[i] = np.unique(table[mask]['rmin_pc'])
    met[i] = np.nanmean(muscat_table[mask]['met_scal'])
sample_table['12+log(O/H)'] = met

mass_co = (sample_table['lco_phangs'].quantity * sample_table['appcor_phangs'].quantity * sample_table['aco10_phangs'].quantity) / 0.65
sample_table['props_mmol'] = mass_co

In [5]:
sample_table_cut = sample_table['name', 'orient_incl', 'orient_posang', 
                                'morph_string', 'dist', 'size_reff', '12+log(O/H)', 'props_mhi', 'props_mmol',  
                                'props_mstar', 'props_sfr']

for i, n in enumerate(sample_table_cut['name']):
    sample_table_cut['name'][i] = n.upper()
    
sample_table_cut['size_reff'] = sample_table_cut['size_reff']*sample_table_cut['dist']*1e6/206268*1e-3*au.kpc/au.arcsec

sample_table_cut['props_mhi'] = np.log10(sample_table_cut['props_mhi'])
sample_table_cut['props_mstar'] = np.log10(sample_table_cut['props_mstar'])
sample_table_cut['props_mmol'] = np.log10(sample_table_cut['props_mmol'])
sample_table_cut['props_sfr'] = np.log10(sample_table_cut['props_sfr'])

In [6]:
sample_table_cut['name']

IC5332
NGC0628
NGC1087
NGC1300
NGC1365
NGC1385
NGC1433
NGC1512
NGC1566
NGC1672
NGC2835


In [7]:
for i, name in enumerate(sample_table_cut['name']):
    if 'IC' in name:
        name = name.split('IC')
        name = f"IC~{name[1]}"
    if 'NGC' in name:
        name = name.split('NGC')
        name = f"NGC~{name[1]}"
    sample_table_cut['name'][i] = name
sample_table_cut

name,orient_incl,orient_posang,morph_string,dist,size_reff,12+log(O/H),props_mhi,props_mmol,props_mstar,props_sfr
,deg,deg,,Mpc,kpc,,solMass,solMass,solMass,solMass / yr
bytes100,float32,float32,bytes100,float32,float64,float64,float32,float32,float32,float32
IC~5332,26.9,74.4,SABc,9.01,3.6078543254406887,8.374076339343995,9.302761,nan,9.667926,-0.38537046
NGC~0628,8.9,20.7,Sc,9.84,3.9026127562200634,8.48488580671925,9.696902,9.431176,10.3412485,0.24364336
NGC~1087,42.9,359.1,Sc,15.85,3.2344145674559313,8.402747040234853,9.100571,9.202594,9.933384,0.11677002
NGC~1300,31.8,278.0,Sbc,18.99,6.525384994279287,8.51570120996781,9.379162,9.399155,10.616569,0.06911268
NGC~1365,55.4,201.1,Sb,19.57,2.78025155622782,8.526218263849604,9.939293,10.257017,10.99019,1.2279408
NGC~1385,44.0,181.3,Sc,17.22,3.372879244478058,8.421211259149379,9.190578,9.226476,9.979256,0.32047546
NGC~1433,28.6,199.7,SBa,18.63,4.303880543758606,8.538555117587507,9.401338,9.293801,10.865968,0.054865655
NGC~1512,42.5,261.9,Sa,18.83,4.758504314774953,8.55483218949262,9.879812,9.123223,10.712756,0.10669651


In [9]:
from numpy import source


names = ['Galaxy',
        #  'RA',
        #  'DEC',
         '$i$',
         'PA',
         'Morph.',
         'Dist.',
         '$R_\mathrm{eff}$',
         '12+log(O/H)',
         '$M_\mathrm{H_I}$',
         '$M_\mathrm{CO}$',
         '$M_\mathrm{star}$',
         'SFR']

formats={
        # 'RA': '%0.3f',
        # 'DEC': '%0.3f',
        'Dist.': '%0.1f', 
        '$R_\mathrm{eff}$': '%0.1f',
        '$M_\mathrm{H_I}$': '%0.1f',
        '$M_\mathrm{CO}$': '%0.1f',
        '$M_\mathrm{star}$': '%0.1f',
#        '$r_\mathrm{eff, min}$': '%0.1f',
        'SFR': '%0.1f',
        '12+log(O/H)': '%0.2f'}

sample_table_cut.write('./tabs/sourceprops.tex', formats=formats, names=names, overwrite=True)
sample_table_cut.write('./tabs/sourceprops.fits', overwrite=True)

In [10]:
sample_table_cut

name,orient_incl,orient_posang,morph_string,dist,size_reff,12+log(O/H),props_mhi,props_mmol,props_mstar,props_sfr
,deg,deg,,Mpc,kpc,,solMass,solMass,solMass,solMass / yr
bytes100,float32,float32,bytes100,float32,float64,float64,float32,float32,float32,float32
IC~5332,26.9,74.4,SABc,9.01,3.6078543254406887,8.374076339343995,9.302761,nan,9.667926,-0.38537046
NGC~0628,8.9,20.7,Sc,9.84,3.9026127562200634,8.48488580671925,9.696902,9.431176,10.3412485,0.24364336
NGC~1087,42.9,359.1,Sc,15.85,3.2344145674559313,8.402747040234853,9.100571,9.202594,9.933384,0.11677002
NGC~1300,31.8,278.0,Sbc,18.99,6.525384994279287,8.51570120996781,9.379162,9.399155,10.616569,0.06911268
NGC~1365,55.4,201.1,Sb,19.57,2.78025155622782,8.526218263849604,9.939293,10.257017,10.99019,1.2279408
NGC~1385,44.0,181.3,Sc,17.22,3.372879244478058,8.421211259149379,9.190578,9.226476,9.979256,0.32047546
NGC~1433,28.6,199.7,SBa,18.63,4.303880543758606,8.538555117587507,9.401338,9.293801,10.865968,0.054865655
NGC~1512,42.5,261.9,Sa,18.83,4.758504314774953,8.55483218949262,9.879812,9.123223,10.712756,0.10669651
